In [ ]:
import yfinance as yf
import pandas as pd

# Define the stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten stock universe to get all tickers
stocks = [ticker for sector in us_stock_universe.values() for ticker in sector]

# Initialize list to store intrinsic values
stock_data_list = []

def calculate_intrinsic_value(ticker):
    try:
        stock = yf.Ticker(ticker)
        eps = stock.info.get("trailingEps", None) or stock.info.get("forwardEps", None)
        growth_rate = stock.info.get("earningsGrowth", None)
        pe_ratio = stock.info.get("trailingPE", None) or stock.info.get("forwardPE", None)
        
        # Skip if EPS or P/E ratio is missing or non-positive
        if eps is None or eps <= 0 or pe_ratio is None or pe_ratio <= 0:
            return None
        
        # Use a default growth rate if missing
        if growth_rate is None or growth_rate <= 0:
            growth_rate = 0.05  # Default 5%
        
        # Clamp growth rate between 0% and 15%
        growth_rate = max(0.00, min(growth_rate, 0.15))
        
        # Calculate intrinsic value using the formula: EPS * (1 + r) * P/E Ratio
        intrinsic_value = eps * (1 + growth_rate) * pe_ratio
        
        return round(intrinsic_value, 2)
    except Exception as e:
        print(f"Error calculating intrinsic value for {ticker}: {e}")
    return None

for stock in stocks:
    intrinsic_value = calculate_intrinsic_value(stock)
    stock_data_list.append({
        "Ticker": stock,
        "Intrinsic Value": intrinsic_value
    })

# Convert to DataFrame
df = pd.DataFrame(stock_data_list)

# Save to CSV
df.to_csv("intrinsic_value.csv", index=False)

print("Intrinsic Value saved to stock_intrinsic_value.csv")
